<a href="https://colab.research.google.com/github/strawndri/corretor_ortografico-nlp/blob/main/Colaboratory/Corretor_ortografico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP - Processamento de Linguagem Natural

A **Processamento de Linguagem Natural (NLP, em inglês)** está relacionada à Inteligência Antificial. Este mecanismo, portanto, atua na conexão entre o ser humano e a máquina.

A interpretação de textos realizada por pessoas, é de certa forma, simples. No entanto, quando este papel cabe à máquina, a situação torna-se mais complicada, uma vez que ela precisa "compreender" inúmeros peculiaridades linguísticas, como coensão, coêrencia, morfologia, semântica, sintaxe, etc.

Alguns exemplos de atividades comuns que utlizam o NLP são: tradução automática, corretor ortográfico, autocomplete (realizado pelo Google, por exemplo), chatbots...

# Importando um Corpus Textual
Obs: Corpus significa, em NLP, uma coleção de documentos

In [7]:
with open('artigos.txt', 'r') as f:
  artigos = f.read()

print(artigos[:500])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:

java 

Suponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou
